#### 1. Import pandas library

In [15]:
import pandas as pd

#### 2. Import mysql.connector


In [2]:
import mysql.connector

#### 2.2. Run the following line


In [20]:
cnx = mysql.connector.connect(user='guest', password='relational', host='relational.fit.cvut.cz', database='stats', use_pure=True)

### Import users table:

#### 3. Initialize cursor (as seen in classes' Jupyter Notebook)

In [21]:
cursor = cnx.cursor()

#### 4. Initialize query  (as seen in classes' Jupyter Notebook) 
#### You want the users table from stats database. (Hint: stats.users)

In [22]:
query = ("SELECT * FROM users")  ## Any query

#### 5. Send query to database  (as seen in classes' Jupyter Notebook)

In [23]:
cursor.execute(query)

#### 6. Create a new dataframe (preferably called users) with the following columns names (as seen in classes' Jupyter Notebook)

In [24]:
columns_users = ['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes', 'AccountId', 'Age', 'ProfileImageUrl']
users_ds = pd.DataFrame(columns = columns_users)

users_ds.head(10)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl


#### 7. Fill the new users dataframe with the data you can get iterating through cursor (as seen in classes' Jupyter Notebook)

In [25]:
for i, data in enumerate(cursor):
    users_ds.loc[i] = data

users_ds.head(23)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,None,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35,None
5,6,457,2010-07-19 19:04:07,Harlan,2014-08-07 19:49:44,http://www.harlan.harris.name,District of Columbia,<ul>\n<li>PhD in CS/AI/Machine Learning/Cognit...,114,47,0,46050,41,None
6,7,429,2010-07-19 19:04:37,Vince,2014-09-10 21:14:12,http://bioinformatics.ucdavis.edu,"Davis, CA",I'm a recent graduate of UC Davis in Economics...,56,20,0,49514,28,None
7,8,6764,2010-07-19 19:04:52,csgillespie,2014-09-09 21:15:08,http://www.mas.ncl.ac.uk/~ncsg3/,"Newcastle, United Kingdom",<p>I'm a statistics lecturer at Newcastle Univ...,1089,604,25,70002,36,None
8,10,121,2010-07-19 19:05:40,Pierre,2014-07-28 18:15:02,http://plindenbaum.blogspot.com,France,Bioinformatician\\nVirology\\nGenetics\\nBiolo...,20,2,0,23234,44,None
9,11,136,2010-07-19 19:06:02,wahalulu,2014-07-23 21:28:18,http://www.linkedin.com/in/marckvaisman,"Washington, DC","<p>data_stuff &lt;- paste('data', c('scientist...",10,10,0,47893,40,None


#### 8. Rename Id column to userId

In [109]:
users_ds = users_ds.rename(columns={"Id": "userId"})

### Import posts table:

#### 9. Initialize cursor (as seen in classes' Jupyter Notebook)

#### 10. Initialize query  (as seen in classes' Jupyter Notebook) 
#### You want the posts table from stats database. (Hint: no hint this time ;) )

#### 11. Send query to database  (as seen in classes' Jupyter Notebook)

#### 12. Create a new dataframe (preferably called posts) with the following columns names (as seen in classes' Jupyter Notebook)

In [125]:
columns_posts = ['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreaionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId', 'LasActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount', 'LastEditorUserId', 'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate', 'OwnerDisplayName', 'LastEditorDisplayName']

In [127]:
posts_ds = pd.DataFrame(columns = columns_posts)

posts_ds.head()

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName


#### 12. Fill the new posts dataframe with the data you can get iterating through cursor (as seen in classes' Jupyter Notebook)

In [128]:
posts_ds = pd.read_csv("/home/inrx/Ironhack/labs/lab-data-cleaning/your-code/posts_table.csv") 

posts_ds.head()

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN


#### 13. Rename Id column to postId and OwnerUserId to userId

In [129]:
posts_ds = posts_ds.rename(columns={"Id": "postId", "OwnerUserId": "userId"})

posts_ds.head()

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN


In [132]:
posts_ds.describe()

,postId,PostTypeId,AcceptedAnswerId,Score,ViewCount,userId,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,ParentId
count,40000.000000,40000.000000,7266.000000,40000.000000,15895.000000,38962.000000,15895.000000,40000.000000,7293.000000,19725.000000,23569.000000
mean,22709.318350,1.636175,23186.498211,4.086500,1206.583202,6079.063087,1.594401,2.011525,3.360346,4635.762180,19986.577666
std,13763.536209,0.594277,14994.186353,6.589488,3872.525910,5224.896435,2.055572,2.673990,7.637988,5751.397127,13752.497021
min,1.000000,1.000000,5.000000,-19.000000,14.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,1.000000
25%,10883.750000,1.000000,10936.500000,1.000000,174.000000,1317.000000,1.000000,0.000000,1.000000,702.000000,7869.000000
50%,22050.500000,2.000000,21794.500000,2.000000,387.000000,4856.000000,1.000000,1.000000,2.000000,2116.000000,18423.000000
75%,33536.500000,2.000000,33589.250000,5.000000,978.000000,9651.000000,2.000000,3.000000,3.000000,7290.000000,30723.000000
max,48325.000000,7.000000,114363.000000,192.000000,175495.000000,55226.000000,136.000000,45.000000,233.000000,53985.000000,48314.000000


#### 14. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [134]:
users_ds = users_ds[["userId", "Reputation","Views","UpVotes","DownVotes"]]
users_ds.head()

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5


In [135]:
posts_ds = posts_ds[["postId", "Score", "userId","ViewCount","CommentCount"]]
posts_ds.head()

,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3


#### 15. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [136]:
users_posts_ds = users_ds.merge(posts_ds)
users_posts_ds.head()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0


In [137]:
users_posts_ds.describe()

,postId,Score,ViewCount,CommentCount
count,38962.000000,38962.000000,15390.000000,38962.000000
mean,22960.799651,4.083081,1196.225991,2.014630
std,13696.932471,6.561843,3742.952815,2.674018
min,1.000000,-19.000000,14.000000,0.000000
25%,11325.250000,1.000000,173.000000,0.000000
50%,22373.500000,2.000000,385.000000,1.000000
75%,33688.500000,5.000000,967.000000,3.000000
max,48325.000000,192.000000,175495.000000,45.000000


#### 16. How many missing values do you have in your merged dataframe? On which columns?

In [138]:
ds_null = users_posts_ds.isnull().sum()
ds_null

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

In [139]:
ds_null [ds_null > 0]  

ViewCount    23572
dtype: int64

There are 23572 missing values from ViewCount DataFrame

#### 17. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [140]:
null_displ = users_posts_ds[(users_posts_ds['ViewCount'].isnull())]
null_displ.head()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0


In [141]:
null_displ

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0
5,-1,1,0,5007,1920,9857,0,NaN,0
6,-1,1,0,5007,1920,9858,0,NaN,0
7,-1,1,0,5007,1920,9860,0,NaN,0
8,-1,1,0,5007,1920,10130,0,NaN,0
9,-1,1,0,5007,1920,10131,0,NaN,0


In [142]:
ds_zeros = users_posts_ds[users_posts_ds == 0.0].count()
ds_zeros

userId              0
Reputation          0
Views            1150
UpVotes          6472
DownVotes       18036
postId              0
Score            4652
ViewCount           0
CommentCount    14983
dtype: int64

In [143]:
users_posts_ds['ViewCount'] = users_posts_ds['ViewCount'].fillna(0)
users_posts_ds

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,0.0,0
1,-1,1,0,5007,1920,8576,0,0.0,0
2,-1,1,0,5007,1920,8578,0,0.0,0
3,-1,1,0,5007,1920,8981,0,0.0,0
4,-1,1,0,5007,1920,8982,0,0.0,0
5,-1,1,0,5007,1920,9857,0,0.0,0
6,-1,1,0,5007,1920,9858,0,0.0,0
7,-1,1,0,5007,1920,9860,0,0.0,0
8,-1,1,0,5007,1920,10130,0,0.0,0
9,-1,1,0,5007,1920,10131,0,0.0,0


Since there are no zeros in the ViewCount collumn I decided to replace the NaN values for 0, since there could be a mistake in the registration of information. It is expectable that a post has 0 ViewCounts. 

#### 18. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [144]:
users_posts_ds.dtypes

userId           object
Reputation       object
Views            object
UpVotes          object
DownVotes        object
postId            int64
Score             int64
ViewCount       float64
CommentCount      int64
dtype: object

In [147]:
users_posts_ds = users_posts_ds.astype('int')

In [148]:
users_posts_ds

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,0,0
1,-1,1,0,5007,1920,8576,0,0,0
2,-1,1,0,5007,1920,8578,0,0,0
3,-1,1,0,5007,1920,8981,0,0,0
4,-1,1,0,5007,1920,8982,0,0,0
5,-1,1,0,5007,1920,9857,0,0,0
6,-1,1,0,5007,1920,9858,0,0,0
7,-1,1,0,5007,1920,9860,0,0,0
8,-1,1,0,5007,1920,10130,0,0,0
9,-1,1,0,5007,1920,10131,0,0,0


In [149]:
users_posts_ds.dtypes

userId          int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
ViewCount       int64
CommentCount    int64
dtype: object